# モンテカルロ法
方策勾配法(2_3)や方策反復法, 価値反復法(2_4_2) では遷移確率 $P$ が既知であることを前提としていた。

確かに迷路ゲームでは, 前の状態と行動が決まれば次の状態が一意に決まるためこの手法で解けた。

今回は, 遷移確率 $P$ がわからない例として, ブラックジャックゲームの学習を行う. 

## 1. ブラックジャックのルール
ブラックシャックはカジノで定番のゲームで、以下のようなルール。

1. ルールの概要
  * トランプを使用する。
  * トランプは無限デッキあると仮定する。（＝カードの出る確率は変化しない）
  * Aは1もしくは11として扱う。
  * 2〜10は数字通り扱う。
  * J, Q, Kは10として扱う。
  * カードの合計が21を越えず、出来るだけ21に近い方が勝ち。（同じなら引き分け）
  
2. プレイの流れ
  * ユーザーにカードが2枚オープンで配られる。
  * ディーラーにカードが1枚はオープン、もう1枚はクローズで配られる。
  * プレイヤーは以下の行動が出来る。
  * ヒット（カードをもう1枚引く）
  * スタンド（カードを引くのを止める）
  * カードの合計が21を越えたら、その時点でプレイヤーの負け。
  * スタンドするか21を越えるまでは、何度でもヒット出来る。
  *  プレイヤーがスタンドを選択したら、ディーラーは伏せていたカードをオープンにし、カードの合計が17以上になるまでカードを引く。
  * カードの合計が21を越えたら、その時点でプレイヤーの勝ち。
  * ディーラーのカードの合計が21以下の場合、カードの合計を比べる。
  * カードの合計が21に近い方の勝ち。同じなら引き分け。

## 2. マルコフ決定過程としてのブラックジャックゲーム
1. 状態集合 $S$ <br>
     $$ S = \{ \text{プレイヤーの状態} \}  \times \{ \text{ディーラーの状態}\} \cup \{ \text{Player勝ち}, \text{Player負け}, \text{引き分け} \}$$
     
     * プレイヤーの状態 <br>
         得点が11以下ならhitしても得点が21を超えない.  <br>
         よって, プレイヤーの手持ちの得点は12以上の状態から始まるものとできる.  <br>
         'A'は1と扱うこともできるし11と扱うこともできる. そこで, 11として扱う'A'が存在するかどうかで場合分け.
         * ( 得点合計 12 ~ 21, 11点として扱う'A'が存在 )
         * ( 得点合計 12 ~ 21, 11点として扱う'A'が存在しない )
          
     * ディーラーの状態 <br>
       　　 1つのカードは見えていない. もう1つのカードは見えていて, [ 'A', 2 ~ 10 ] の場合がある.

2. 行動集合 $A$ <br>
    $$ A = \{ \text{hit}, \text{stand} \} $$
    
3. 報酬関数 $r$ <br>
    $$ r(s, a, \text{Player勝ち} ) = 1,\ r(s, a, \text{Player負け} ) = -1,\ r(s, a, \text{引き分け} ) = 0 \ ( \forall s \in S,\ \forall a \in A)$$
    
4. 遷移関数 $P$ <br>
    hitかstandか行動を取った後にどの状態に行くかは確率的に変化する. この確率が未知とする.
    
5. 割引率 $ \gamma $ <br>
    $$\gamma = 1.0 $$


In [2]:
# ブラックジャックゲームで遊ぶ. 
# blackjackゲームのclassはblackjack.pyを参照.
from blackjack import *
game = BlackJack()
while True:
    game.show_status()
    s = input("[h] : hit, [s] : stand \n")
    if s == "h":
        game.player_hit()
    elif s == "s":
        game.player_stand()
    else:
        print("Wrong Input")
        continue
    if game.finish:
        break

game.show_status()
result = game.result()
if result > 0:
    print("Player Win!")
elif result < 0:
    print("Player Lose!")
else:
    print("Draw.")

--------------------
Player (total : 12) [ 10, 2 ]
Dealer (total : ??) [ 10, 8, ?? ]
--------------------
[h] : hit, [s] : stand 
h
--------------------
Player (total : 22) [ 10, 2, J ]
Dealer (total : 18) [ 10, 8 ]
--------------------
Player Lose!


## 3. モンテカルロ法の概要

### 3.1 方策評価
方策反復法や価値反復法では, 状態価値関数 $V^{\pi}$ を求めることで方策 $\pi$ の評価を行なっていた.
    
今回も同様に $V^{\pi},\ Q^{\pi}$ を求めたい. しかし, そもそもベルマン方程式が連立方程式として解けない.

さらに $P$ がわからない時 $Q^{\pi}$ から $V^{\pi}$ は求まるが, $V^{\pi}$ から $Q^{\pi}$ は求まらない. (参照 : 2_4_2 の 4. )

なので, $Q^{\pi}$ を定義に戻って求めたいと考える.
$$ Q^{\pi} (s,a) = \mathbb{E}[ \sum_{n=t}^{\infty} \gamma^{n-t} r_n  ; s_t = s,\ a_t = a ] $$

期待値は「理論上の平均」であることから, シミュレーションをして得られた収益の平均で期待値を推定する.
$$ \mathbb{E}[ \sum_{n=t}^{\infty} \gamma^{n-t} r_n  ; s_t = s,\ a_t = a ] \simeq \dfrac{1}{M} \sum_{m=1}^M \sum_{n=t}^{\infty} \gamma^{n-t} r_{m,n}  \ (s_{m, t} = s,\ a_{m,t} = a )$$

> 大数の(強)法則 : 確率変数$X_1,\ X_2,\ \dots, X_n$ が独立同分布に従っていれば, 
$$ P( \lim_{n \to \infty} \dfrac{1}{n} \sum_{i=1}^n X_i = \mu ) = 1 $$

どのサンプルを採用するかによって2つの手法がある.
* 初回訪問 : 
    1つのエピソード列中で, 初めて観測されたときの状態の収益だけを扱う.$ \forall t' < t,\  (s_{t'}, a_{t'}) \neq (s,a) \land (s_t, a_t) = (s,a) $ の時, $R_t = \sum_{n = t}^{\infty} \gamma^{n - t} r_{n}$ をサンプルとして採用.
* 逐次訪問 : 観測された全ての状態の収益をサンプルとして扱う. <br>
    $ (s_t, a_t) = (s, a) $ の時, $R_t = \sum_{n = t}^{\infty} \gamma^{n - t} r_{n}$ をサンプルとして採用.

### 3.2 方策改善
方策改善は, 2_4_2 と同じように greedy または $\epsilon$-greedy で更新を行う. 再記述すると, 
* greedy : 
    $$ \pi'(s, a) \leftarrow \begin{cases} 1 \quad ( a = \underset{a'\in A}{\operatorname{argmax}} Q^{\pi}(s, a') ) \\ 0 \quad ( otherwise ) \end{cases} $$

* $\epsilon$-greedy : 
    $$ \pi'(s, a) \leftarrow \begin{cases} 1 - (|A| - 1) \epsilon \quad &( a = \underset{a'\in A}{\operatorname{argmax}} Q^{\pi}(s, a') ) \\ \epsilon & \quad ( otherwise ) \end{cases} $$
 
### 3.3 乱数シュミレーションの問題点

シュミレーションする時, 全ての状態が観測され, 各状態から様々な行動を取った結果の収益を平均化したい.
しかし, 決定論的な方策を取っていると1つの状態からは同じ行動しか取らず, 学習がうまくいかない。<br>
( greedyで方策改善すると決定論的な方策になる )

これを解決する方法がいくつか考えられている.

* 開始点探査の仮定をおき, 任意の状態行動対を開始点としてシミュレーションする. &rarr; モンテカルロES法
開始点探査の仮定とは, 「任意の状態行動対 $(s, a) $ が出現する確率が0ではない」という仮定.
方策が決定論的であっても任意の状態行動対 $(s, a) $ が生成される.

* 方策を決定論的でないソフトな方策 $\epsilon$-greedyに変更する. &rarr; 方策オン型モンテカルロ法

* シュミレーション時の方策( 挙動方策 ) と本番用の方策 ( 推定方策 ) を別にする. &rarr; 方策オフ型モンテカルロ法


### 3.4 モンテカルロ-ES法
ここでは, 方策評価は初回訪問の方式で行う. <br>
価値反復法のように $Q^{\pi}$ の更新と $\pi$ の更新を交互に行う.

<img src="https://yoheitaonishi.com/wp-content/uploads/2018/10/es.png" width=500 >

In [3]:
#%load_ext autoreload
import numpy as np
from tqdm import tqdm
from collections import OrderedDict

# 状態集合の定義
# S : 状態集合, S_to_num : 各状態に整数を割り振った対応表
S_to_num = OrderedDict()
state_num = 0
for player_score in range(12, 22):
    for ace_num in [ 0, 1 ]:
        for dealer_card in range(1, 11):
            S_to_num[ ( player_score, ace_num, dealer_card)] =  state_num
            state_num += 1
S_to_num["PlayerWin"] = state_num
S_to_num["PlayerLose"] = state_num + 1
S_to_num["Draw"] = state_num + 2
S = list(S_to_num.keys())

# 行動集合 A, A_to_num : 各行動に整数を割り振った対応表
A_to_num = { "hit" : 0, "stand" : 1}
A = [ "hit" , "stand" ]

# 報酬関数
def reward(s1, a, s2):
    if s2 =="PlayerWin":
        return 1
    elif s2 == "PlayerLose":
        return -1
    elif s2 == "Draw":
        return 0
    else:
        return 0
    
# 割引率
gamma = 1.0

# (s_0, a_0) から 方策piに従ってゲームが進行した時のエピソード列を返す.
def generate_episode(pi, s_0, a_0):
    game = BlackJack(s_0=s_0)
    episode = [ [s_0, a_0] ]
    
    # 初期行動
    if a_0 == "hit":
        game.player_hit()
    elif a_0 == "stand":
        game.player_stand()
    # ゲームの状態を取得.
    s = game.state()
    episode[-1].append(reward(s_0, a_0, s))
    
    while not game.finish:
        # 次の行動を選択.
        a = np.random.choice(A, p=pi[S_to_num[s] , :])
        if a == "hit":
            game.player_hit()
        elif a == "stand":
            game.player_stand()
        # 次の状態を取得.
        next_s = game.state()
        # 報酬を取得.
        r = reward(s, a, next_s)
        # エピソード列に追加.
        episode.append([ s, a, r ])
        s = next_s
    return episode

# 与えられたpiを確率化する.
def to_prob(pi):
    n, m = pi.shape
    for i in range(n):
        pi[i] = pi[i, :] / np.sum(pi[i, :])
    return pi

def greedy(Q):
    argmax = np.argmax(Q, axis=1)
    new_pi = np.zeros_like(Q)
    for s_raw in S:
        s = S_to_num[s_raw]
        new_pi[s , argmax[s]] = 1.0
    return new_pi
    
# モンテカルロES法
def Monte_Carlo_ES():
    # Q, pi を適当に初期化.
    Q = np.random.rand(  len(S), len(A) )
    pi = to_prob(np.random.rand(  len(S), len(A) ))
    
    # 最後のゲーム終了状態は考える必要なし. ( 放置していても良いが, 念のため. )
    Q[-3:, :] = 0
    pi[-3:, :] = 0
    
    #  Returns[s][a] : [ 収益のリスト ]
    Returns = [ [ [] for _ in range(len(A)) ] for _ in range(len(S)) ]
    iterations = 50000
    for roop in tqdm(range(iterations)):
        # 初期状態
        s_0 = np.random.choice(S)
        # 終状態から実行しても意味がない
        if s_0 == "PlayerWin" or s_0 == "PlayerLose" or s_0 == "Draw":
            continue
        # 初期行動
        a_0 = np.random.choice(A)
        
        # エピソード生成.
        episode = generate_episode(pi, s_0, a_0)
        
        # 既にエピソード列の中に状態行動対( s, a )が登場したかをもつ. Falseで初期化.
        visited = np.zeros((len(S), len(A)))
        
        R_list = [0] * len (episode)
        R_t = 0
        # 各時刻における収益を計算
        for i, (s_t, a_t, r_t) in enumerate(reversed(episode)):
            s, a = S_to_num[s_t], A_to_num[a_t]
            R_t += gamma*r_t
            R_list[i] = R_t
        R_list.reverse()

        for i, (s_t, a_t, r_t) in enumerate(episode):
            # 状態と行動を数字に変換
            s, a = S_to_num[s_t], A_to_num[a_t]
            # 収益の値.
            R_t = R_list[i]
            # 初回訪問の場合のみ扱う.
            if not visited[s,a]:
                # Returns[s][a]に(s,a)以降のエピソード列で得られる収益(報酬の和)を記録.
                Returns[s][a].append(R_t)
                # QはReturns[s][a]の平均で更新
                # ※ len(Returns[s][a]) の値を記録する配列 Returns_Length[s][a] を定義すると, 
                # ※ Q[s][a] <- ( Q[s][a] * (Returns_Length[s][a] - 1) + R_t ) / Returns_Length[s][a] と更新が可能.
                Q[s, a] = sum(Returns[s][a]) / len(Returns[s][a])
                visited[s, a] = True
        # piの更新.
        pi = greedy(Q)
            
    return Q, pi, Returns
  
Q, pi_es, Returns = Monte_Carlo_ES()

100%|██████████| 50000/50000 [00:22<00:00, 2235.04it/s]


In [4]:
# 最適方策の表示
def show_policy(S, pi):
    for s in S:
        if s == "PlayerWin" or s == "PlayerLose" or s == "Draw":
            continue
        s_n = S_to_num[s]
        if pi[s_n, 0] < pi[s_n, 1]:
            print("State : {0}, best-pi : {1}".format(s,  "stand" ))
        if pi[s_n, 0] >= pi[s_n, 1]:
            print("State : {0}, best-pi : {1}".format(s, "hit"))
            
show_policy(S, pi_es)

State : (12, 0, 1), best-pi : hit
State : (12, 0, 2), best-pi : hit
State : (12, 0, 3), best-pi : hit
State : (12, 0, 4), best-pi : stand
State : (12, 0, 5), best-pi : hit
State : (12, 0, 6), best-pi : stand
State : (12, 0, 7), best-pi : hit
State : (12, 0, 8), best-pi : hit
State : (12, 0, 9), best-pi : hit
State : (12, 0, 10), best-pi : hit
State : (12, 1, 1), best-pi : hit
State : (12, 1, 2), best-pi : hit
State : (12, 1, 3), best-pi : hit
State : (12, 1, 4), best-pi : hit
State : (12, 1, 5), best-pi : hit
State : (12, 1, 6), best-pi : hit
State : (12, 1, 7), best-pi : hit
State : (12, 1, 8), best-pi : hit
State : (12, 1, 9), best-pi : hit
State : (12, 1, 10), best-pi : hit
State : (13, 0, 1), best-pi : hit
State : (13, 0, 2), best-pi : hit
State : (13, 0, 3), best-pi : hit
State : (13, 0, 4), best-pi : hit
State : (13, 0, 5), best-pi : hit
State : (13, 0, 6), best-pi : stand
State : (13, 0, 7), best-pi : hit
State : (13, 0, 8), best-pi : hit
State : (13, 0, 9), best-pi : hit
State 

## 3.5 方策オン型モンテカルロ法

方策を確率的なソフト方策 $\epsilon$-greedyに変更することで, 任意の状態行動対 $(s, a)$ が生成される.

<img src="https://yoheitaonishi.com/wp-content/uploads/2018/10/e%E3%82%BD%E3%83%95%E3%83%88%E6%96%B9%E7%AD%96.png" width=500>

#### [余談] greedy と $\epsilon$-greedy の関係
ここでは「任意の状態行動対 $(s, a)$ が生成する」目的で $\epsilon$-greedy を導入したが, 2_4_2 で述べた方策反復法や価値反復法の方策改善アルゴリズムとして, greedy の代わりに $\epsilon$-greedy を用いても良い.

greedyなアルゴリズムでは, 初期値によっては局所的最適解に陥る.
<img src="https://www.ffri.jp/assets/images/blog/201306/pic1.png" width=500>

そこで, 確率的なゆらぎを導入して, 大域的最適解を求める.

学習が進むにつれて $\epsilon$ の値 ( 図ではTemprature ) を小さくしていく.
<img src="https://upload.wikimedia.org/wikipedia/commons/d/d5/Hill_Climbing_with_Simulated_Annealing.gif" width=500>

最適化において, greedyなアルゴリズムを確率的なアルゴリズムに拡張することはよくある.
- (組み合わせ最適化) 山登り法 &rarr; 焼きなまし法(アニーリング)
- (連続最適化) 最急降下法 &rarr; 確率的勾配法

この拡張のメリットは, 大域的最適解が初期値によらず発見可能.デメリットは, 収束に時間がかかること.

ちなみに凸関数( 山/谷が1つ )であれば局所的最適解と大域的最適解が一致するので特有の最適化手法があって速い. ([凸最適化](https://myenigma.hatenablog.com/entry/2016/11/01/135447))

In [5]:
def epsilon_greedy(Q, epsilon):
    # argmaxに 1 - epsilon + epsilon / |A| , それ以外に epsilon / |A|
    argmax = np.argmax(Q, axis=1)
    new_pi = np.ones_like(Q) * (epsilon / len(A))
    for s_raw in S:
        s = S_to_num[s_raw]
        new_pi[s , argmax[s]] += 1 - epsilon
    return new_pi

# 初回訪問方策オン型モンテカルロ法, 本質的には, ES法からpiの更新部分が変更されているだけ.
def on_policy_Monte_Carlo(epsilon = 0.3):
    # Q, pi を適当に初期化.
    Q = np.random.rand(  len(S), len(A) )
    pi = to_prob(np.random.rand(  len(S), len(A) ))
    
    # 最後のゲーム終了状態は考える必要なし. ( 放置していても良いが, 念のため. )
    Q[-3:, :] = 0
    pi[-3:, :] = 0
    # Returnsリストの長さを持つことにする.
    Returns_Length = [ [0]*len(A) for _ in range(len(S)) ]
    iterations = 50000
    for roop in tqdm(range(iterations)):
        # 初期状態
        s_0 = np.random.choice(S)
        # 終状態から実行しても意味がない
        if s_0 == "PlayerWin" or s_0 == "PlayerLose" or s_0 == "Draw":
            continue
        # 初期行動
        a_0 = np.random.choice(A)
        
        # エピソード生成.
        episode = generate_episode(pi, s_0, a_0)
        
        # 既にエピソード列の中に状態行動対( s, a )が登場したかをもつ. Falseで初期化.
        visited = np.zeros((len(S), len(A)))
        
        R_list = [0] * len (episode)
        R_t = 0
        # 各時刻における収益を計算
        for i, (s_t, a_t, r_t) in enumerate(reversed(episode)):
            s, a = S_to_num[s_t], A_to_num[a_t]
            R_t += gamma*r_t
            R_list[i] = R_t
        R_list.reverse()
    
        for i, (s_t, a_t, r_t) in enumerate(episode):
            # 状態と行動を数字に変換
            s, a = S_to_num[s_t], A_to_num[a_t]
            # 収益の値.
            R_t = R_list[i]
            # 初回訪問の場合のみ扱う.
            if not visited[s,a]:
                Returns_Length[s][a] += 1
                # この式と等価: 
                # Q[s,a] = (Q[s,a] * (Returns_Length[s][a] - 1) + R_t) / Returns_Length[s][a]
                Q[s,a] = Q[s,a] + (R_t - Q[s,a]) / (Returns_Length[s][a])
                visited[s, a] = True
        # piの更新. 
        pi = epsilon_greedy(Q, epsilon)
        # epsilonを小さく
        epsilon /= 2
        
    return Q, pi, Returns
                
Q, pi_on, Returns = on_policy_Monte_Carlo(epsilon=0.3)

100%|██████████| 50000/50000 [00:19<00:00, 2524.29it/s]


In [6]:
# 最適方策の表示
show_policy(S, pi_on)

State : (12, 0, 1), best-pi : hit
State : (12, 0, 2), best-pi : stand
State : (12, 0, 3), best-pi : hit
State : (12, 0, 4), best-pi : hit
State : (12, 0, 5), best-pi : stand
State : (12, 0, 6), best-pi : stand
State : (12, 0, 7), best-pi : hit
State : (12, 0, 8), best-pi : hit
State : (12, 0, 9), best-pi : hit
State : (12, 0, 10), best-pi : hit
State : (12, 1, 1), best-pi : hit
State : (12, 1, 2), best-pi : hit
State : (12, 1, 3), best-pi : hit
State : (12, 1, 4), best-pi : hit
State : (12, 1, 5), best-pi : hit
State : (12, 1, 6), best-pi : stand
State : (12, 1, 7), best-pi : hit
State : (12, 1, 8), best-pi : hit
State : (12, 1, 9), best-pi : hit
State : (12, 1, 10), best-pi : hit
State : (13, 0, 1), best-pi : hit
State : (13, 0, 2), best-pi : hit
State : (13, 0, 3), best-pi : hit
State : (13, 0, 4), best-pi : stand
State : (13, 0, 5), best-pi : stand
State : (13, 0, 6), best-pi : hit
State : (13, 0, 7), best-pi : hit
State : (13, 0, 8), best-pi : hit
State : (13, 0, 9), best-pi : stan

## 3.6 方策オフ型モンテカルロ法
シュミレーション時の方策( 挙動方策 ) と本番用の方策 ( 推定方策 ) を別で用意することで, 
任意の状態行動対 $(s, a)$ が生成される. ただし, 挙動方策は必ずsoftな方策, 推定方策はどちらでも良い. 

### 3.6.1 方策評価の導出
挙動方策 $\pi_b$ を用いて行動価値関数 $Q^{\pi_b}$ を求めるが, 欲しいのは推定方策 $\pi$ の行動価値関数 $ Q^{\pi} $ である.

そこで, $\pi_b$ でシュミレーションした結果から, $\pi$ でシュミレーション場合の結果を近似的に求める.

方策 $\pi$ に従うエピソード列 $\{ (s_t, a_t) \}_{l \leqq t \leqq u}$ が生成される確率は
$$ \begin{eqnarray} 
    & &P(s_l, a_l, s_{l+1}) \pi(s_{l+1}, a_{l+1}) P(s_{l+1}, a_{l+1}, s_{l+2}) \cdots \pi(s_{u}, a_u) P(s_{u}, a_{u}, s_{u+1}) \\
    &=& P(s_l, a_l, s_{l+1}) \prod_{t=l+1}^u \pi(s_{t+1}, a_{t+1}) P(s_{t+1}, a_{t+1}, s_{t+2}) 
\end{eqnarray} $$ 
方策 $\pi_b$ に従ったシュミレーションでエピソード列 $\{ (s_t, a_t) \}_{l \leqq t \leqq u}$ が $N$ 回観測された時, 
方策$\pi$ に従うシュミレーションで同エピソード列の観測回数 $M$ は, 
$$ \begin{eqnarray}
M &=& N \dfrac{P(s_l, a_l, s_{l+1}) \prod_{t=l+1}^u \pi(s_{t+1}, a_{t+1}) P(s_{t+1}, a_{t+1}, s_{t+2})}{P(s_l, a_l, s_{l+1}) \prod_{t=l+1}^u \pi_b(s_{t+1}, a_{t+1}) P(s_{t+1}, a_{t+1}, s_{t+2})} \\
&=& N \prod_{t = l+1}^u \dfrac{ \pi(s_t, a_t) }{ \pi_b(s_t, a_t) }
\end{eqnarray} $$
ちなみに, 推定方策 $\pi$ をgreedyで更新する場合には, $\pi(s,a)$ は決定論的な方策となり, 常に1であるから
$$ M = N \prod_{t = l+1}^u \dfrac{1}{ \pi_b(s_t, a_t) } $$

以前のモンテカルロ法からの変更点を示す.
まずわかりやすさのため, $Returns[s][a]$ をリストの合計値 $ReturnsValue[s][a]$ とリストの長さ$ReturnsLength[s][a]$ でもつように変更する.

次にある試行のステップ $t$ において, $(s_t, a_t) = (s,a)$ なら
$$ \begin{eqnarray}
ReturnsValue[s][a] &\leftarrow& ReturnsValue[s][a] + R_t \prod_{t = l+1}^u \dfrac{ \pi(s_t, a_t) }{ \pi_b(s_t, a_t) } \\
ReturnsLength[s][a] &\leftarrow& ReturnsLength[s][a] + 1 \prod_{t = l+1}^u \dfrac{ \pi(s_t, a_t) }{ \pi_b(s_t, a_t) } 
\end{eqnarray} $$

### 3.6.2 方策改善
挙動方策は必ずsoftな方策, 推定方策はどちらでも良い. 

### 3.6.3 全体のアルゴリズム

以下では, 推定方策 $\pi$ をgreedyで更新しているので, $ \prod_{t = l+1}^u \dfrac{1}{ \pi_b(s_t, a_t) } $ を使っていることに注意. 

$ReturnsValue$ も 1つ前の $Q$ と $ ReturnsLength$ を使って逆算されているので注意.
<img src="https://i.stack.imgur.com/Xi0vX.png" width=500 >



In [7]:
def greedy(Q):
    argmax = np.argmax(Q, axis=1)
    new_pi = np.zeros_like(Q)
    for s_raw in S:
        s = S_to_num[s_raw]
        new_pi[s , argmax[s]] = 1.0
    return new_pi

def epsilon_greedy(Q, epsilon):
    # argmaxに 1 - epsilon + epsilon / |A| , それ以外に epsilon / |A|
    argmax = np.argmax(Q, axis=1)
    new_pi = np.ones_like(Q) * (epsilon / len(A))
    for s_raw in S:
        s = S_to_num[s_raw]
        new_pi[s , argmax[s]] += 1 - epsilon
    return new_pi

# 初回訪問方策オフ型モンテカルロ法.　
# 挙動方策 pi_a : epsilon-greedyで更新. 推定方策 pi : Qからgreedyに更新.
def off_policy_Monte_Carlo(epsilon=0.5):
    
    #############  Initialize  ###################
    # Q, を適当に初期化.
    Q = np.random.rand(  len(S), len(A) )
    # 推定方策をQを元に初期化 (決定論的な方策)
    pi = greedy(Q) 
    # 挙動方策を適当に初期化 ( ソフトな方策 )
    pi_b = to_prob(np.random.rand(  len(S), len(A) ))
    # Returnsの代わりに, Returnsのリストの長さを持つことにする.
    Returns_Length = [ [0]*len(A) for _ in range(len(S)) ]
    
    ############# シュミレーションをiterations回数やる #################
    iterations = 50000
    for roop in tqdm(range(iterations)):
        # 初期状態の選択
        s_0 = np.random.choice(S)
        # 終状態から実行しても意味がない
        if s_0 == "PlayerWin" or s_0 == "PlayerLose" or s_0 == "Draw":
            continue
        # 初期行動の選択
        a_0 = np.random.choice(A)
        
        # 挙動方策 pi_b を使って, エピソード生成.
        episode = generate_episode(pi_b, s_0, a_0)
        
        # 既にエピソード列の中に状態行動対( s, a )が登場したかをもつ. Falseで初期化.
        visited = np.zeros((len(S), len(A)))
        # 収益を計算
        R_list = [0] * len (episode)
        # pi / pi_b の総積を計算
        W_list = [0] * len(episode)
        R_t = 0
        W_t = 1
        # pi[s, a]が0になってしまう場所を記録.
        init_t = 0
        
        # 各時刻における収益とpi/pi_bの総積を前計算.
        for i, (s_t, a_t, r_t) in enumerate(reversed(episode)):
            s, a = S_to_num[s_t], A_to_num[a_t]
            # pi[s,a] == 0.0 なら W_tが以降ずっと0なので終了
            if pi[s, a] != 1.0:
                init_t = len(episode) - i
                break
            W_t *= (1 / pi_b[s, a])
            R_t += gamma * r_t
            R_list[i] = R_t
            W_list[i] = W_t
        W_list.reverse()
        R_list.reverse()
        
        ########## 方策評価　############
        # 先頭からエピソードをみる ( 初回モンテカルロ法 )
        for i, (s_t, a_t, r_t) in enumerate(episode[init_t:]):
            # 状態と行動を数字に変換
            s, a = S_to_num[s_t], A_to_num[a_t]
            idx = init_t + i
            # ステップtでの pi / pi_b の総積
            W_t = W_list[idx]
            # ステップtでの収益
            R_t = R_list[idx]
            # 初回訪問の場合のみ扱う.
            if not visited[s,a]:
                Returns_Length[s][a] += 1 * W_t
                # 次の式と同値 :
                # Q[s,a] = (Q[s,a] * (Returns_Length[s][a] - 1*W_t) + R_t*W_t) / Returns_Length[s,a]
                Q[s,a] = Q[s,a] + ((R_t - Q[s,a])*W_t) / (Returns_Length[s][a])
                visited[s, a] = True
        
       ########## 方策改善　############
        # 推定方策の更新.
        pi = greedy(Q)
        # 行動方策の更新 
        pi_b = epsilon_greedy(Q, epsilon)
        # epsilonを小さく.
        epsilon /= 2
        
    return Q, pi, Returns_Length

Q, pi_off, Returns_Length = off_policy_Monte_Carlo(0.5)

100%|██████████| 50000/50000 [00:22<00:00, 2203.02it/s]


In [8]:
show_policy(S, pi_off)

State : (12, 0, 1), best-pi : hit
State : (12, 0, 2), best-pi : hit
State : (12, 0, 3), best-pi : hit
State : (12, 0, 4), best-pi : stand
State : (12, 0, 5), best-pi : stand
State : (12, 0, 6), best-pi : hit
State : (12, 0, 7), best-pi : stand
State : (12, 0, 8), best-pi : hit
State : (12, 0, 9), best-pi : hit
State : (12, 0, 10), best-pi : hit
State : (12, 1, 1), best-pi : hit
State : (12, 1, 2), best-pi : stand
State : (12, 1, 3), best-pi : stand
State : (12, 1, 4), best-pi : stand
State : (12, 1, 5), best-pi : hit
State : (12, 1, 6), best-pi : hit
State : (12, 1, 7), best-pi : stand
State : (12, 1, 8), best-pi : hit
State : (12, 1, 9), best-pi : hit
State : (12, 1, 10), best-pi : hit
State : (13, 0, 1), best-pi : hit
State : (13, 0, 2), best-pi : stand
State : (13, 0, 3), best-pi : hit
State : (13, 0, 4), best-pi : hit
State : (13, 0, 5), best-pi : hit
State : (13, 0, 6), best-pi : stand
State : (13, 0, 7), best-pi : hit
State : (13, 0, 8), best-pi : stand
State : (13, 0, 9), best-p